### TODO

* PHASE 1 Get team data from standings 
* PHASE 1 - Get and merge player data (below)
* PHASE 1 - Get player info / photo link
* PHASE 2 - Get match data / once and increment

###  LINKS

In [8]:
player_uri = 'https://www.euroleaguebasketball.net/euroleague/stats/expanded/?size=1000&viewType=advanced&seasonCode=E2022&statisticMode=perGame&seasonMode=Single&sortDirection=ascending&statistic='
real_madrid = 'https://www.euroleaguebasketball.net/euroleague/teams/real-madrid/statistics/mad/?season=2022-23&phase=All%20phases'
teams = 'https://www.euroleaguebasketball.net/euroleague/teams'
game_center = 'https://www.euroleaguebasketball.net/euroleague/game-center/'
standings = 'https://www.euroleaguebasketball.net/euroleague/standings/'

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

### GET PLAYER DATA

In [3]:
def get_driver():
    service = Service(r'driver/chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [6]:
def extract_df(stats_link):
    driver = get_driver()
    driver.get(stats_link)
    time.sleep(5)
#     print(driver.page_source)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
    rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

    header_list = []
    for cell in rows[0]:
    #     print(cell.text, end= ' ')
        header_list.append(cell.text)

    data = []
    for row in rows:
        row_data = []
        for cell in row.find_all('div', {'role':'cell'}):
            if cell.find('a'):
                span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
    #             print(span.text, end=' ')
                row_data.append(span.text)
            else:
    #             print(cell.text, end=' ')
                row_data.append(cell.text)
    #     print('\n')
        data.append(row_data)
    return pd.DataFrame(data[1:], columns=header_list)

In [5]:
extract_df(player_uri)

,#,Player,Team,GP,Min,eFG%,TS%,OREB%,DREB%,REB%,AST/TO,AST-R,TO-R,2PTA-R,3PTA-R,FT-RT
0,1,ARTURS KURUCS,BKN,21,6:53,42.6%,43%,1.6%,2.4%,2%,1.4,30.6%,22.1%,15.3%,30.6%,7.4%
1,2,MATTHEW STRAZEL,ASM,20,7:20,45.2%,47.8%,0.8%,1.8%,1.2%,1.2,22.2%,18.8%,23.9%,29%,25.8%
2,3,SANDER RAIESTE,BKN,18,6:59,50%,48.2%,7.5%,11.2%,9.3%,1,14.3%,14.3%,40.1%,28.7%,8.3%
3,4,KAROLIS LUKOSIUNAS,ZAL,19,7:03,58.8%,58.8%,2.6%,4.7%,3.6%,1.5,13.6%,9.1%,6.8%,70.5%,0%
4,5,RAFI MENCO,MTA,20,5:37,44.4%,47.7%,9%,10.8%,9.9%,1.5,15.7%,10.4%,26.1%,44.4%,11.1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,204,NIKOLA MIROTIC,BAR,22,25:18,60.5%,65.1%,9.1%,14.1%,11.7%,0.8,8.2%,9.7%,37.3%,31.8%,43%
204,205,VASILIJE MICIC,EFS,29,31:29,53.7%,59.2%,1.3%,11%,6.3%,2,24.9%,12.8%,23.5%,31.2%,32%
205,206,WILL CLYBURN,EFS,32,33:15,52.2%,57.1%,3.9%,17.7%,11%,1.3,11.8%,8.8%,39.1%,31%,30%
206,207,MATHIAS LESSORT,PAR,32,30:35,68.9%,72.6%,14.3%,16.5%,15.4%,1,12.1%,12.4%,57.3%,0.3%,70.8%


### GET TEAM DATA

In [84]:
def extract_team_df(stats_link):
    driver = get_driver()
    driver.get(stats_link)
    time.sleep(5)
#     print(driver.page_source)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
    rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

    header_list = []
    for cell in rows[0].find_all('div', attrs={'class':'complex-stat-table_cell__1lxC7'}):
        span = cell.find('span', attrs={'class':'complex-stat-table_tabletHeadName__1Jn7u'})
        if span:
            header_list.append(span.text.replace('*','').strip())
        else:
            header_list.append(cell.find('span').text.replace('*','').strip())

    data = []
    for row in rows:
        row_data = []
        for cell in row.find_all('div', {'role':'cell'}):
            if cell.find('a'):
                span = cell.find('span', attrs={'class':'complex-stat-table_mainClubName__3IMZJ'})
    #             print(span.text, end=' ')
                row_data.append(span.text)
            else:
    #             print(cell.text, end=' ')
                row_data.append(cell.text)
    #     print('\n')
        data.append(row_data)
    return pd.DataFrame(data[1:], columns=header_list)

In [85]:
extract_team_df(standings)

,Position,Club,GP,Won,Lost,Win%,PTS+,PTS-,+/-,H,A,OT,L10
0,1,Olympiacos Piraeus *,32,23,9,71.9%,2692,2413,+279,13-3,10-6,0-0,8-2
1,2,FC Barcelona *,32,22,10,68.8%,2562,2425,+137,12-4,10-6,1-1,8-2
2,3,Real Madrid *,32,22,10,68.8%,2712,2513,+199,12-4,10-6,1-1,7-3
3,4,AS Monaco *,32,21,11,65.6%,2646,2574,+72,13-3,8-8,2-1,7-3
4,5,Maccabi Playtika Tel Aviv,32,19,13,59.4%,2674,2589,+85,14-2,5-11,0-0,8-2
5,6,Fenerbahce Beko Istanbul,32,18,14,56.2%,2641,2580,+61,11-5,7-9,2-0,5-5
6,7,Partizan Mozzart Bet Belgrade,32,18,14,56.2%,2706,2616,+90,11-5,7-9,0-2,7-3
7,8,Cazoo Baskonia Vitoria-Gasteiz,32,17,15,53.1%,2721,2650,+71,13-3,4-12,2-0,5-5
8,9,Zalgiris Kaunas,32,17,15,53.1%,2440,2479,-39,12-4,5-11,0-0,5-5
9,10,Anadolu Efes Istanbul,32,16,16,50%,2627,2561,+66,10-6,6-10,0-2,5-5


### GET PLAYER INFO

In [86]:
belinelli = 'https://www.euroleaguebasketball.net/euroleague/players/marco-belinelli/bcn/'

In [88]:
driver = get_driver()
driver.get(belinelli)
time.sleep(5)
#     print(driver.page_source)
soup = BeautifulSoup(driver.page_source,'lxml')

In [91]:
spans = soup.find('h1', attrs={'class':'hero-info_fullName__IDuVE'}).find_all('span')
name = ''
for span in spans:
    name += f' {span.text}'
name = name.strip()
name

'MARCO BELINELLI'

In [92]:
soup.find('h2', attrs={'class':'hero-info_teamName__3g98M'}).text

'Virtus Segafredo Bologna'

In [93]:
info = {}
for li in soup.find('ul', attrs={'class','hero-info_dataList__2BmgP'}):
    info[li.find('span').text] = li.find('b').text
info

{'Nationality': 'Italy', 'Height': '195', 'Born': '25 Mar 1986'}

In [101]:
img = soup.find('div', attrs={'class':'player-hero_imageWrap__161BQ cover-contain_ofCover__3K_IS'}).find('img')
pic_link = img['data-srcset'].split('?')[0]
pic_link

'https://media-cdn.incrowdsports.com/f13366de-a8da-4503-a64b-420104fd1777.png'

In [ ]:
table = soup.find(name='div', attrs='stats-table_table__2BoHU')
rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

In [4]:
# link = '/euroleague/teams/alba-berlin/roster/ber/'
def get_stats_uri(link): 
    url = 'https://www.euroleaguebasketball.net/'
    acc = '?season=2022-23&phase=All%20phases#accumulated'
    avg = '?season=2022-23&phase=All%20phases#average'
    temp_split = link.strip('/').rsplit('/', 2)
    uri = url + temp_split[0] + '/statistics/' + temp_split[2]
    uri_avg = uri + '/' + avg
    uri_acc = uri + '/' + acc
    return uri_acc, uri_avg

In [8]:
def get_team_links(teams):
    source = requests.get(teams)
    soup = BeautifulSoup(source.text, 'lxml')
    team_list = soup.find('ul', {'class':'teams-list_list__3M_EG'})
    team_links = {}
    for li in team_list.find_all('li'):
        name = li.find('span', {'class':'teams-card_name__3HBk_'}).text
        link = li.find('a')['href']
        team_links[name] = get_stats_uri(link)
    return team_links

In [6]:
def get_driver():
    service = Service(r'driver/chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [7]:
def extract_df(stats_link):
    driver = get_driver()
    driver.get(stats_link)
    time.sleep(5)
#     print(driver.page_source)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
    rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

    header_list = []
    for cell in rows[0]:
    #     print(cell.text, end= ' ')
        header_list.append(cell.text)

    data = []
    for row in rows:
        row_data = []
        for cell in row.find_all('div', {'role':'cell'}):
            if cell.find('a'):
                span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
    #             print(span.text, end=' ')
                row_data.append(span.text)
            else:
    #             print(cell.text, end=' ')
                row_data.append(cell.text)
    #     print('\n')
        data.append(row_data)
    return pd.DataFrame(data[1:], columns=header_list)

In [23]:
#get match links for round
driver = get_driver()
driver.get('https://www.euroleaguebasketball.net/euroleague/game-center/?round=2&season=E2022')
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'lxml')
match_links = []
match_lists = soup.find_all('ul', {'class':'game-center-group_gameGroup__2lpzG'})
for match_list in match_lists:
    for li in match_list.find_all('li'):
        home_div = li.find('div', {'class':'game-card-view_teamHomeWrapper__1ra78'})
        home = home_div.find('span', {'class':'game-card-view_name__3qJw7'}).text
        away_div = li.find('div', {'class':'game-card-view_teamAwayWrapper__23L8i'})
        away = away_div.find('span', {'class':'game-card-view_name__3qJw7'}).text
        link = li.find('a', {'class':'game-card-view_linkWrap__u3Tea'})['href']
        match_links.append((home, away, link))
match_links

[('Crvena Zvezda Meridianbet Belgrade',
  'Panathinaikos Athens',
  '/euroleague/game-center/2022-23/crvena-zvezda-meridianbet-belgrade-panathinaikos-athens/E2022/11/'),
 ('AS Monaco',
  'Anadolu Efes Istanbul',
  '/euroleague/game-center/2022-23/as-monaco-anadolu-efes-istanbul/E2022/10/'),
 ('Virtus Segafredo Bologna',
  'FC Bayern Munich',
  '/euroleague/game-center/2022-23/virtus-segafredo-bologna-fc-bayern-munich/E2022/13/'),
 ('FC Barcelona',
  'Real Madrid',
  '/euroleague/game-center/2022-23/fc-barcelona-real-madrid/E2022/12/'),
 ('Olympiacos Piraeus',
  'Zalgiris Kaunas',
  '/euroleague/game-center/2022-23/olympiacos-piraeus-zalgiris-kaunas/E2022/15/'),
 ('Fenerbahce Beko Istanbul',
  'Maccabi Playtika Tel Aviv',
  '/euroleague/game-center/2022-23/fenerbahce-beko-istanbul-maccabi-playtika-tel-aviv/E2022/14/'),
 ('Cazoo Baskonia Vitoria-Gasteiz',
  'Partizan Mozzart Bet Belgrade',
  '/euroleague/game-center/2022-23/cazoo-baskonia-vitoria-gasteiz-partizan-mozzart-bet-belgrade/E20

In [ ]:
driver.get('/euroleague/game-center/2022-23/crvena-zvezda-meridianbet-belgrade-panathinaikos-athens/E2022/11/#boxscore')
time.sleep
soup = BeautifulSoup(driver.page_source,'lxml')
table = soup.find(name='div', attrs='//*[@id="main"]/div/div/div[2]/div/div[3]/div[3]/div[2]/div/div[1]/div/div[1]')
rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

header_list = []
for cell in rows[0]:
#     print(cell.text, end= ' ')
    header_list.append(cell.text)

data = []
for row in rows:
    row_data = []
    for cell in row.find_all('div', {'role':'cell'}):
        if cell.find('a'):
            span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
#             print(span.text, end=' ')
            row_data.append(span.text)
        else:
#             print(cell.text, end=' ')
            row_data.append(cell.text)
#     print('\n')
    data.append(row_data)
pd.DataFrame(data[1:], columns=header_list)

In [19]:
li

<li class="game-center-group_li__3buHh"><article class="game-card-view_gameCard__1nUzg"><a class="game-card-view_linkWrap__u3Tea" href="/euroleague/game-center/2022-23/real-madrid-fenerbahce-beko-istanbul/E2022/279/"><span class="visually-hidden_wrap__1CCX0">game Fenerbahce Beko Istanbul vs Real Madrid</span></a><div class="game-card-view_marker__1lub_ game-card-view__hidden__2ujZY">Group Regular Season</div><div class="game-card-view_gameCardInner__1P2tg"><div class="game-card-view_leftSideWrapper__3i_7a game-card-view__mobile__1D777"><div><time class="game-card-view_time__2pzFM" datetime="2023-03-29T18:45:00.000Z">29 March 2023, 18:45</time></div><span class="game-card-view_innerSeason__3CnA5">Regular Season</span></div><div class="game-card-view_side__3Z3CI"><div class="game-card-view_leftSideWrapper__3i_7a"><div><time class="game-card-view_time__2pzFM" datetime="2023-03-29T18:45:00.000Z">29 March 2023, 18:45</time></div><span class="game-card-view_innerSeason__3CnA5">Regular Season

In [ ]:
def get_match_links(game_center):
    match_links = {}
    for round in range(1, 35):
        url = game_center + f'?round={round}&season=E2022'
        source = requests.get(teams)
        match_links[round] = 
    soup = BeautifulSoup(source.text, 'lxml')
    team_list = soup.find('ul', {'class':'teams-list_list__3M_EG'})
    team_links = {}
    for li in team_list.find_all('li'):
        name = li.find('span', {'class':'teams-card_name__3HBk_'}).text
        link = li.find('a')['href']
        team_links[name] = get_stats_uri(link)
    return team_links

In [55]:
driver = get_driver()
driver.get()

<class 'str'>


In [57]:
i = 0
for team, links in team_links.items():
    if i<1:
        acc_fn = team + '-acc.csv'
        acc = extract_df(links[0])
        acc.to_csv(acc_fn)
#         avg_fn = team + '-avg.csv'
#         avg = extract_df(links[1])
#         avg.to_csv(avg_fn)
acc

KeyboardInterrupt: 

In [31]:
temp_split

['euroleague/teams/alba-berlin/roster', 'ber']

In [21]:
team_list.find_all('li')[5].find('a')['href']

'/euroleague/teams/ea7-emporio-armani-milan/roster/mil/'

In [71]:
service = Service(r'driver/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
driver.get(real_madrid)

In [50]:
driver = get_driver()
driver.get('https://www.euroleaguebasketball.net/euroleague/teams/alba-berlin/statistics/ber/?season=2022-23&phase=All%20phases#accumulated')
driver.page_source

'<html class="html" lang="en-GB" data-whatinput="initial" data-whatintent="initial"><head><meta name="google-site-verification" content="K3MkZpH_0Ns1bU9ImO-WhalPSFDPuN69TZqrFLvJyhc"><script>\n                                const localStorageString = window.localStorage.getItem("cookieState");\n                                let state;\n                                if (localStorageString) {\n                                    try {\n                                        state = JSON.parse(localStorageString);\n                                    } catch (e) {\n                                        console.log(e);\n                                    }\n                                }\n                                if (state && !state.isAllowedPerformance) {\n                                    window["ga-disable-UA-185286673-1"] = true;\n                                }\n                            </script><script>if (window.location.search.search(\'headers=false\') >= 0

In [51]:
soup = BeautifulSoup(driver.page_source,'lxml')
table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

In [52]:
rows

[<div class="complex-stat-table_row__1P6us" role="rowheader"><div class="complex-stat-table_cell__1lxC7 complex-stat-table__head__3wslB" role="columnheader"><span class="">#</span></div><div class="complex-stat-table_cell__1lxC7 complex-stat-table__head__3wslB complex-stat-table__sticky__TGKMh complex-stat-table__shadow__3uUYS complex-stat-table__left__2eKns" role="columnheader"><span class="">Player</span></div><div class="complex-stat-table_cell__1lxC7 complex-stat-table__head__3wslB" role="columnheader"><button class="complex-stat-table_headBtn__2kIEG complex-stat-table__asc__3xWa-" type="button"><span class="">GP</span></button></div><div class="complex-stat-table_cell__1lxC7 complex-stat-table__head__3wslB" role="columnheader"><button class="complex-stat-table_headBtn__2kIEG complex-stat-table__asc__3xWa-" type="button"><span class="">GS</span></button></div><div class="complex-stat-table_cell__1lxC7 complex-stat-table__head__3wslB" role="columnheader"><button class="complex-stat-

In [120]:
soup = BeautifulSoup(driver.page_source,'lxml')
table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

header_list = []
for cell in rows[0]:
#     print(cell.text, end= ' ')
    header_list.append(cell.text)
    
data = []
for row in rows:
    row_data = []
    for cell in row.find_all('div', {'role':'cell'}):
        if cell.find('a'):
            span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
#             print(span.text, end=' ')
            row_data.append(span.text)
        else:
#             print(cell.text, end=' ')
            row_data.append(cell.text)
#     print('\n')
    data.append(row_data)
pd.DataFrame(data[1:], columns=header_list)

,#,Player,GP,GS,Min,PTS,2P%,3P%,FT%,OR,DR,TR,AST,STL,TO,BLK,BLKA,FC,FD,PIR
0,1,NIGEL WILLIAMS-GOSS,19,16,14:20,6.5,63.2%,38.1%,68.4%,0.2,1.4,1.6,1.8,0.7,0.9,0.0,0.1,1.6,1.3,6.9
1,2,FABIEN CAUSEUR,22,7,13:51,4.5,40.9%,34.1%,77.3%,0.3,1.0,1.3,1.4,0.5,0.8,0.0,0.1,1.7,1.3,3.6
2,3,RUDY FERNANDEZ,9,0,17:49,4.0,50%,26.3%,100%,0.2,1.4,1.7,1.9,0.7,0.6,0.2,0.2,1.9,1.1,3.6
3,4,ALBERTO ABALDE,21,8,14:15,2.4,34.8%,22.6%,63.6%,0.5,1.6,2.0,1.6,0.3,0.6,0.0,0.1,1.7,0.9,2.7
4,5,ADAM HANGA,15,7,10:51,3.3,45.8%,31.6%,62.5%,0.1,1.5,1.7,1.4,0.5,0.9,0.1,0.0,2.1,1.4,3.3
5,6,MARIO HEZONJA,27,10,17:46,9.0,55.3%,43.6%,84.8%,0.2,2.3,2.4,1.1,0.5,1.4,0.1,0.1,1.7,1.4,7.9
6,7,SERGIO RODRIGUEZ,28,3,13:45,4.0,44.7%,29.5%,78.9%,0.1,1.4,1.5,4.1,0.4,1.0,0.0,0.1,1.6,1.0,5.5
7,8,GABRIEL DECK,23,17,26:15,12.3,63.1%,31.7%,91.5%,1.5,3.7,5.2,2.0,0.6,1.6,0.1,0.3,2.1,3.0,15.3
8,9,VINCENT POIRIER,28,1,14:01,5.7,62%,50%,71.7%,1.5,2.5,4.0,0.5,0.2,0.9,1.0,0.1,2.2,1.8,8.1
9,10,PETR CORNELIE,26,15,16:07,6.8,75.6%,40%,81.8%,0.6,3.0,3.6,0.4,0.4,0.5,0.2,0.0,1.4,0.9,8.0


In [115]:
len(header_list)

20

In [109]:
data

['1',
 'NIGEL WILLIAMS-GOSS',
 '19',
 '16',
 '14:20',
 '6.5',
 '63.2%',
 '38.1%',
 '68.4%',
 '0.2',
 '1.4',
 '1.6',
 '1.8',
 '0.7',
 '0.9',
 '0.0',
 '0.1',
 '1.6',
 '1.3',
 '6.9',
 '2',
 'FABIEN CAUSEUR',
 '22',
 '7',
 '13:51',
 '4.5',
 '40.9%',
 '34.1%',
 '77.3%',
 '0.3',
 '1.0',
 '1.3',
 '1.4',
 '0.5',
 '0.8',
 '0.0',
 '0.1',
 '1.7',
 '1.3',
 '3.6',
 '3',
 'RUDY FERNANDEZ',
 '9',
 '0',
 '17:49',
 '4.0',
 '50%',
 '26.3%',
 '100%',
 '0.2',
 '1.4',
 '1.7',
 '1.9',
 '0.7',
 '0.6',
 '0.2',
 '0.2',
 '1.9',
 '1.1',
 '3.6',
 '4',
 'ALBERTO ABALDE',
 '21',
 '8',
 '14:15',
 '2.4',
 '34.8%',
 '22.6%',
 '63.6%',
 '0.5',
 '1.6',
 '2.0',
 '1.6',
 '0.3',
 '0.6',
 '0.0',
 '0.1',
 '1.7',
 '0.9',
 '2.7',
 '5',
 'ADAM HANGA',
 '15',
 '7',
 '10:51',
 '3.3',
 '45.8%',
 '31.6%',
 '62.5%',
 '0.1',
 '1.5',
 '1.7',
 '1.4',
 '0.5',
 '0.9',
 '0.1',
 '0.0',
 '2.1',
 '1.4',
 '3.3',
 '6',
 'MARIO HEZONJA',
 '27',
 '10',
 '17:46',
 '9.0',
 '55.3%',
 '43.6%',
 '84.8%',
 '0.2',
 '2.3',
 '2.4',
 '1.1',
 '0.5',
 '1.4'